In [1]:
import pandas as pd
import numpy as np

#### 13~16

In [61]:
import pandas as pd
import numpy as np

def experiment(sample_size, tau):
    rng = np.random.default_rng()
    # Generate random array x and compute y
    x = rng.uniform(-0.5, 0.5, sample_size)
    x.sort()
    def my_sign(x):
        return np.sign(np.where(x != 0, x, -1))
    y = my_sign(x)

    def flip_labels(y, tau):
        if tau > 0:
            random_vals = np.random.random(size=y.shape)
            flip_indices = random_vals <= tau
            y_flipped = y.copy()
            y_flipped[flip_indices] *= -1
            return y_flipped
        else:
            return y


    # Flip y to -y with probability tau
    y = flip_labels(y, tau)

    # Generate new random array x and compute y
    x_new = rng.uniform(-0.5, 0.5, sample_size)
    x_new.sort()
    y_new = my_sign(x_new)
    y_new = flip_labels(y_new, tau)

    # Define the hypothesis h
    def h(x, s, theta):
        return s * np.sign(x - theta)

    # Compute possible values of theta
    unique_x = np.unique(x)
    diffs = np.diff(unique_x)
    if len(diffs) == 0:
        theta_vals = [-np.inf, np.inf]
    else:
        theta_vals = np.hstack(([-np.inf], unique_x[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

    # Compute Ein(h) for all combinations of s and theta
    misclassified = []
    for s in [-1, 1]:
        for theta in theta_vals:
            predictions = h(x, s, theta)
            misclassified.append((predictions != y).sum())

    # Compute the minimum misclassification error and corresponding s and theta
    min_error_idx = np.argmin(misclassified)
    min_error = misclassified[min_error_idx]
    min_error_s = [-1, 1][min_error_idx // len(theta_vals)]
    min_error_theta = theta_vals[min_error_idx % len(theta_vals)]

    # Return the hypothesis with the minimum Ein as g; break ties by smallest s * theta
    g = h(x, min_error_s, min_error_theta)
    for s in [-1, 1]:
        for theta in theta_vals:
            if s == min_error_s and theta == min_error_theta:
                continue
            error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()
            if misclassified[error_idx] == min_error and s * theta < min_error_s * min_error_theta:
                g = h(x, s, theta)
                min_error_s = s
                min_error_theta = theta

    Ein_g = min_error / len(x)
    Eout_g = np.mean(h(x_new, min_error_s, min_error_theta) != y_new)
    return Eout_g - Ein_g

n_trials = 10000
sample_size = 128
tau = 0.2   # Set probability of flipping
results = [experiment(sample_size, tau) for i in range(n_trials)]
mean_diff = np.mean(results)

print("Mean of Eout(g, tau) - Ein(g) over", n_trials, "trials:", mean_diff)


<ipython-input-61-8d982ff13f0b>:64: RuntimeWarning: invalid value encountered in subtract
  error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()


Mean of Eout(g, tau) - Ein(g) over 10000 trials: 0.0143875


#### 17、18

In [67]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
X_train = df_train.iloc[:, :-1].to_numpy()
y_train = df_train.iloc[:, -1].to_numpy()
sample_size_train = df_train.shape[0]

df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
X_test = df_test.iloc[:, :-1].to_numpy()
y_test = df_test.iloc[:, -1].to_numpy()
sample_size_test = df_test.shape[0]
X_train_idx = X_train.shape[0]
y_train_idx = y_train.shape[0]
print(X_train_idx)
print(y_train_idx)

192
192


##### 先小修13~16的code

In [66]:
import pandas as pd
import numpy as np

def experiment():

    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    # Define the hypothesis h
    def h(X, s, theta):
        return s * np.sign(X[:, 0] - theta)

    def compute_theta_vals(X_train):
        theta_vals = []
        for col in range(X_train.shape[1]):
            unique_x = np.unique(X_train[:,col])
            diffs = np.diff(unique_x)
            if len(diffs) == 0:
                theta_vals_col = [-np.inf, np.inf]
            else:
                theta_vals_col = np.hstack(([-np.inf], unique_x[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))
            theta_vals.append(theta_vals_col)
        return theta_vals


    theta_vals = compute_theta_vals(X_train)

    # Compute Ein(h) for all combinations of s and theta
    misclassified = []
    for s in [-1, 1]:
        for theta in theta_vals:
            predictions = h(X_train, s, theta)
            misclassified.append((predictions != y_train).sum())

    # Compute the minimum misclassification error and corresponding s and theta
    min_error_idx = np.argmin(misclassified)
    min_error = misclassified[min_error_idx]
    min_error_s = [-1, 1][min_error_idx // len(theta_vals)]
    min_error_theta = theta_vals[min_error_idx % len(theta_vals)]

    # Return the hypothesis with the minimum Ein as g; break ties by smallest s * theta
    g = h(X_train, min_error_s, min_error_theta)
    for s in [-1, 1]:
        for theta in theta_vals:
            if s == min_error_s and theta == min_error_theta:
                continue
            error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()
            if misclassified[error_idx] == min_error and s * theta < min_error_s * min_error_theta:
                g = h(x, s, theta)
                min_error_s = s
                min_error_theta = theta

    Ein_g = min_error / len(x)
    Eout_g = np.mean(h(x_test, min_error_s, min_error_theta) != y_test)


    return Ein_g, Eout_g, Eout_g - Ein_g

Ein_g, Eout_g, Eout_Ein_diff = experiment()
print(f"Ein(g): {Ein_g:.2f}")
print(f"Eout(g): {Eout_g:.2f}")
print(f"Eout(g) - Ein(g): {Eout_Ein_diff:.2f}")


ValueError: operands could not be broadcast together with shapes (192,) (187,) 

In [ ]:
import pandas as pd
import numpy as np

def decision_stump(X, y):
    n, d = X.shape
    misclassified = np.zeros((2*d, n))
    thetas = np.zeros((2*d, d))
    for i in range(d):
        idx = np.argsort(X[:,i])
        X_i = X[idx,i]
        y_i = y[idx]
        thetas[2*i, i] = X_i[0] - 1.0
        thetas[2*i+1, i] = X_i[-1] + 1.0
        for j in range(n-1):
            thetas[2*i+j+2, i] = (X_i[j] + X_i[j+1]) / 2.0
        for s in [-1, 1]:
            for j in range(2*n):
                y_pred = s * np.sign(X[:,i] - thetas[j,i])
                misclassified[j,:] += y_pred != y
    min_error_idx = np.argmin(misclassified)
    min_error = misclassified.flatten()[min_error_idx]
    min_error_s = [-1, 1][min_error_idx // n // d]
    min_error_thetas = thetas[min_error_idx // n % d]
    g = lambda x: min_error_s * np.sign(x - min_error_thetas)
    return g, min_error / n

def experiment():
    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    # Extract features and labels
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    # Extract features and labels
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    # Find the best decision stump using the training data
    g, Ein = decision_stump(X_train, y_train)

    # Compute the test error Eout
    Eout = np.mean(g(X_test) != y_test)

    return Ein_g, Eout_g, Eout_g - Ein_g

Ein_g, Eout_g, Eout_Ein_diff = experiment()
print(f"Ein(g): {Ein_g:.2f}")
print(f"Eout(g): {Eout_g:.2f}")
print(f"Eout(g) - Ein(g): {Eout_Ein_diff:.2f}")

In [20]:
import pandas as pd
import numpy as np

def experiment():
    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    def h(x, s, theta):
        predictions = s * np.sign(x - theta)
        return predictions

    # Compute possible values of theta
    unique_X_train = np.unique(X_train)
    diffs = np.diff(unique_X_train)
    if len(diffs) == 0:
        theta_vals = [-np.inf, np.inf]
    else:
        theta_vals = np.hstack(([-np.inf], unique_X_train[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

    # Compute Ein(h) and Eout(h) for all thresholds
    Ein_vals = []
    Eout_vals = []
    for theta in theta_vals:
        Ein = np.mean(np.not_equal(h(X_train, 1, theta), y_train))
        Eout = np.mean(np.not_equal(h(X_test, 1, theta), y_test))
        Ein_vals.append(Ein)
        Eout_vals.append(Eout)

    # Find the threshold that minimizes Eout(h)
    min_Eout_idx = np.argmin(Eout_vals)
    min_Eout = Eout_vals[min_Eout_idx]
    min_Eout_theta = theta_vals[min_Eout_idx]

    # Compute the corresponding Ein(h) and Eout(g)
    min_Ein = Ein_vals[min_Eout_idx]
    Eout_g = min_Eout
    Ein_g = min_Ein
    return Ein_g, Eout_g, Eout_g - Ein_g

Ein_g, Eout_g, Eout_Ein_diff = experiment()
print(f"Ein(g): {Ein_g:.2f}")
print(f"Eout(g): {Eout_g:.2f}")
print(f"Eout(g) - Ein(g): {Eout_Ein_diff:.2f}")


ValueError: operands could not be broadcast together with shapes (192,10) (192,) 

##### 有跑出來，但是是錯的

In [68]:
import pandas as pd
import numpy as np

def experiment():
    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    def h(x, s, theta):
        predictions = np.zeros_like(x)
        for i in range(len(x)):
            predictions[i] = s * np.sign(x[i] - theta)
        return predictions

    # Compute possible values of theta
    unique_X_train = np.unique(X_train)
    diffs = np.diff(unique_X_train)
    if len(diffs) == 0:
        theta_vals = [-np.inf, np.inf]
    else:
        theta_vals = np.hstack(([-np.inf], unique_X_train[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

    # Compute Ein(h) for all values of s and theta
    min_error = np.inf
    min_error_s = None
    min_error_theta = None
    min_error_i = None

    for s in [-1, 1]:
        for i, theta in enumerate(theta_vals):
            predictions = h(X_train, s, theta)
            error = np.sum(np.not_equal(predictions.reshape(-1,1), y_train))

            if error < min_error:
                min_error = error
                min_error_s = s
                min_error_theta = theta
                min_error_i = i

    Ein_g = np.mean(np.not_equal(h(X_train, min_error_s, min_error_theta).reshape(-1, 1), y_train))
    Eout_g = np.mean(np.not_equal(h(X_test, min_error_s, min_error_theta).reshape(-1, 1), y_test))

    return Ein_g, Eout_g, Eout_g - Ein_g

Ein_g, Eout_g, Eout_Ein_diff = experiment()
print(f"Ein(g): {Ein_g:.2f}")
print(f"Eout(g): {Eout_g:.2f}")
print(f"Eout(g) - Ein(g): {Eout_Ein_diff:.2f}")


Ein(g): 0.33
Eout(g): 0.42
Eout(g) - Ein(g): 0.09


##### best of best algorithm

In [77]:
import pandas as pd
import numpy as np

def experiment1():
    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]
    num_features = X_train.shape[1]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    best_hypothesis = None
    min_error = np.inf

    for i in range(num_features):
        unique_X_train = np.unique(X_train[:, i])
        diffs = np.diff(unique_X_train)
        if len(diffs) == 0:
            theta_vals = [-np.inf, np.inf]
        else:
            theta_vals = np.hstack(([-np.inf], unique_X_train[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

        for s in [-1, 1]:
            for theta in theta_vals:
                predictions = s * np.sign(X_train[:, i] - theta)
                error = np.sum(np.not_equal(predictions, y_train))

                if error < min_error:
                    min_error = error
                    best_hypothesis = (i, s, theta)
                elif error == min_error and i < best_hypothesis[0]:
                    best_hypothesis = (i, s, theta)

    def h(x):
        i, s, theta = best_hypothesis
        return s * np.sign(x[:, i] - theta)

    Ein_g = np.mean(np.not_equal(h(X_train), y_train))
    Eout_g = np.mean(np.not_equal(h(X_test), y_test))

    return Ein_g, Eout_g, Eout_g - Ein_g

Ein_g1, Eout_g1, Eout_Ein_diff1 = experiment1()
print(f"Ein(g1): {Ein_g1:.5f}")
print(f"Eout(g1): {Eout_g1:.5f}")
print(f"Eout(g1) - Ein(g1): {Eout_Ein_diff1:.5f}")


Ein(g1): 0.02604
Eout(g1): 0.07812
Eout(g1) - Ein(g1): 0.05208


#### 19、20

In [76]:
import pandas as pd
import numpy as np

def experiment2():
    df_train = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)
    X_train = df_train.iloc[:, :-1].to_numpy()
    y_train = df_train.iloc[:, -1].to_numpy()
    sample_size_train = df_train.shape[0]
    num_features = X_train.shape[1]

    df_test = pd.read_csv('ML_HW2_data_test.dat', sep="\t", header=None)
    X_test = df_test.iloc[:, :-1].to_numpy()
    y_test = df_test.iloc[:, -1].to_numpy()
    sample_size_test = df_test.shape[0]

    best_hypotheses = [None] * num_features
    min_errors = np.inf * np.ones(num_features)

    for i in range(num_features):
        unique_X_train = np.unique(X_train[:, i])
        diffs = np.diff(unique_X_train)
        if len(diffs) == 0:
            theta_vals = [-np.inf, np.inf]
        else:
            theta_vals = np.hstack(([-np.inf], unique_X_train[:-1][diffs != 0] + diffs[diffs != 0] / 2, [np.inf]))

        for s in [-1, 1]:
            for theta in theta_vals:
                predictions = s * np.sign(X_train[:, i] - theta)
                error = np.mean(np.not_equal(predictions, y_train))

                if error < min_errors[i]:
                    min_errors[i] = error
                    best_hypotheses[i] = (i, s, theta)
                elif error == min_errors[i] and i < best_hypotheses[i][0]:
                    best_hypotheses[i] = (i, s, theta)

    best_hypothesis = best_hypotheses[np.argmax(min_errors)]

    def h(x):
        i, s, theta = best_hypothesis
        return s * np.sign(x[:, i] - theta)

    Ein_g2 = np.mean(np.not_equal(h(X_train), y_train))
    Eout_g2 = np.mean(np.not_equal(h(X_test), y_test))

    return Ein_g, Eout_g, Eout_g - Ein_g

Ein_g2, Eout_g2, Eout_Ein_diff2 = experiment2()
print(f"Ein(g2): {Ein_g2:.5f}")
print(f"Eout(g2): {Eout_g2:.5f}")
print(f"Eout(g2) - Ein(g2): {Eout_Ein_diff2:.5f}")


Ein(g): 0.32812
Eout(g): 0.42188
Eout(g) - Ein(g): 0.09375


In [81]:
print(f"delta_Ein = {Ein_g2 - Ein_g1:.5f}")
print(f"delta_Eout = {Eout_g2 - Eout_g1:.5f}")

delta_Ein =  0.30208
delta_Eout =  0.34375
